In [ ]:
!pip install contractions rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a4cfef6836bc9f7893da8c302e79103c485dbaf4bb7b252b674e30c06e30dbd0
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import chardet
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import numpy as np
import pandas as pd
import contractions
import unicodedata
import re
import networkx as nx
from rouge_score import rouge_scorer
import networkx as nx
from bs4 import BeautifulSoup

#for reproducibility purpose
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
dataset=pd.read_excel('366_ARPs_for_extracting_Issue_Solution_Pairs.xlsx')

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#Load the dataset
data = pd.read_excel('366_ARPs_for_extracting_Issue_Solution_Pairs.xlsx')

# Preprocessing the texts in ARPs

In [ ]:
# Applying heuristic technique to reduce noice in the data
def clean_dataset(text):
    if not isinstance(text, str):
        return text

    soup = BeautifulSoup(text, "html.parser")

    for a in soup.find_all('a'):
        a.replace_with('[external-link]')

    for img in soup.find_all('img'):
        img.replace_with('[figure]')

    for code in soup.find_all('code'):
        code.replace_with('[code-snippet]')

    for table in soup.find_all('table'):
        table.replace_with('[table]')

    clean_text = soup.get_text()

    return clean_text

# Apply the function to 'Question_body' and 'Answer_body' columns
dataset['Question_body_cleaned'] = dataset['Question_body'].apply(clean_dataset)
dataset['Answer_body_cleaned'] = dataset['Answer_body'].apply(clean_dataset)

In [ ]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    sentences = sent_tokenize(text)

    stop_words = set(ENGLISH_STOP_WORDS)

    punctuation = set(string.punctuation)
    extra_special_characters = set(["''", '``', '##', '>>', '<<', 'e', 'g', 'eg', 'cant', 'cannot', 'isnt', 'would', 'could', 'doesnt', 'hasnt', 'thanks', '-', ')', '\n'])
    special_characters = extra_special_characters.union(punctuation)

    processed_sentences = []

    for sentence in sentences:
        cleaned_words = re.findall(r'\b\w+\b', sentence.lower())
        cleaned_words = [word for word in cleaned_words if word not in stop_words]
        cleaned_words = [word for word in cleaned_words if word not in special_characters]
        cleaned_words = [word for word in cleaned_words if not word.isdigit()]
        cleaned_words = [word for word in cleaned_words if word.isalpha()]

        lemmatized_words = [lemmatizer.lemmatize(word) for word in cleaned_words]

        processed_sentences.append(cleaned_words)

    return sentences, processed_sentences


In [ ]:
# Preprocess the Question_body and Answer_body columns
dataset['processed_question'] = dataset['Question_body_cleaned'].apply(preprocess_text)
dataset['processed_answer'] = dataset['Answer_body_cleaned'].apply(preprocess_text)

dataset[['processed_question', 'processed_answer']].head()

,processed_question,processed_answer
0,([I need help with the architecture pattern I ...,([So first of all what we're talking about her...
1,([which part of file structure we should do pr...,"([For the purpose of clean architecture, the f..."
2,([I am building c# .NET 4.8.1 MVC web applicat...,([ There is no easy and reliable way to do thi...
3,([We have a mobile application that we scale a...,([There is not a simple answer to this questio...
4,([I'm trying to properly design an application...,([Determining the source of the information is...


Building the Similarity Matrix



In [ ]:
# Build similarity matrix function
def build_similarity_matrix(processed_sentences):
    sentences = [" ".join(sentence) for sentence in processed_sentences]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return similarity_matrix

# Apply similarity matrix building
dataset['question_similarity_matrix'] = dataset['processed_question'].apply(lambda x: build_similarity_matrix(x[1]))
dataset['answer_similarity_matrix'] = dataset['processed_answer'].apply(lambda x: build_similarity_matrix(x[1]))

dataset[['question_similarity_matrix', 'answer_similarity_matrix']].head()

,question_similarity_matrix,answer_similarity_matrix
0,"[[1.0, 0.0, 0.09473376964436592, 0.0, 0.0, 0.0...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1456608..."
1,"[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0865651075502646, 0.034802321376..."
2,"[[1.0, 0.07028081026664326, 0.0690588812162663...","[[1.0, 0.1773322077340031, 0.1316482812253033,..."
3,"[[1.0000000000000002, 0.0, 0.11193188131429124...","[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"[[0.9999999999999999, 0.097745674227839, 0.033...","[[1.0, 0.11062087717781531, 0.1267026034121509..."


Applying LexRank Algorithm

In [ ]:
# LexRank algorithm
def lex_rank(similarity_matrix, threshold=0.1):
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph, alpha=0.85, tol=1e-4)
    return scores

# Apply LexRank to the similarity matrices for both questions and answers
dataset['question_scores'] = dataset['question_similarity_matrix'].apply(lex_rank)
dataset['answer_scores'] = dataset['answer_similarity_matrix'].apply(lex_rank)

dataset[['question_scores', 'answer_scores']].head()

,question_scores,answer_scores
0,"{0: 0.05772693326335411, 1: 0.0565801559096096...","{0: 0.05962657218806743, 1: 0.0627088255257249..."
1,"{0: 0.16666666666666666, 1: 0.1666666666666666...","{0: 0.166980743256598, 1: 0.14285714285714285,..."
2,"{0: 0.10642897108007064, 1: 0.095578860268711,...","{0: 0.06221471772859618, 1: 0.0770324390702145..."
3,"{0: 0.11203119243579776, 1: 0.1111111111111111...","{0: 0.125, 1: 0.11967235605098596, 2: 0.122728..."
4,"{0: 0.06256619698292634, 1: 0.0613359775168389...","{0: 0.1504041132564807, 1: 0.14645061652608898..."


# Generating the Summary/Issue-Solution pairs

In [ ]:
# Generate summary function
def generate_summary(original_sentences, scores, num_sentences=6):
    ranked_sentence_indices = np.argsort([scores[i] for i in range(len(scores))])[::-1]

    summary = []
    for i in range(min(num_sentences, len(original_sentences))):
        summary.append(original_sentences[ranked_sentence_indices[i]])

    return " ".join(summary)

# GGenerate summaries or issue–solution pairs for both questions and answers.
dataset['Issue_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_question'][0], x['question_scores']), axis=1)
dataset['Solution_Extracted'] = dataset.apply(lambda x: generate_summary(x['processed_answer'][0], x['answer_scores']), axis=1)

In [ ]:
# Display the summaries or issue–solution pairs
summaries = dataset[['Question_title', 'Issue_Extracted', 'Solution_Extracted']]
summaries.head()

,Question_title,Issue_Extracted,Solution_Extracted
0,Separation of Students and Users in NestJS Mic...,"Considering this, creating users, contacts, an...",If something cannot be done with one repositor...
1,Flutter Clean Architecture,Regards I hope I explain my question clearly. ...,"For the purpose of clean architecture, the fro..."
2,Correct .NET Architecture for long running asy...,If they execute it immediately the web page sh...,Cancellation of the operation can also be mark...
3,Architecture for white-label mobile apps with ...,We have a mobile application that we scale as ...,Now imagine if you have a pure UI layer that g...
4,Implementing Data Source Selection Logic in Cl...,"Then, as I understand it, there are several ap...",They are different concerns BECAUSE they have ...


In [ ]:
# Save the extracted issue-solution pairs to a new Excel file
file_path = '/content/LexRank_Extracted_Issue_Solution.xlsx'

dataset.to_excel(file_path, index=False, engine='openpyxl')

# Evaluate the extracted issue-solution pairs using Precsion, Recall, and F1

In [ ]:
df=pd.read_excel('LexRank_Extracted_Issue_Solution.xlsx')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

def evaluate_summaries_at_sentence_level(df, ref_col, gen_col):
    precision_list = []
    recall_list = []
    f1_list = []

    for index, row in df.iterrows():
        ref_summary = row[ref_col]
        gen_summary = row[gen_col]

        if pd.isna(ref_summary) or pd.isna(gen_summary):
            continue

        ref_sentences = nltk.sent_tokenize(ref_summary)
        gen_sentences = nltk.sent_tokenize(gen_summary)

        ref_sentences_set = set(ref_sentences)
        gen_sentences_set = set(gen_sentences)

        precision = len(ref_sentences_set & gen_sentences_set) / len(gen_sentences_set) if gen_sentences_set else 0
        recall = len(ref_sentences_set & gen_sentences_set) / len(ref_sentences_set) if ref_sentences_set else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    metrics_df = pd.DataFrame({
        'precision': precision_list,
        'recall': recall_list,
        'f1': f1_list
    })

    return metrics_df

# Load DataFrame containing the extracted issue-solution pairs
df = pd.read_excel('LexRank_Extracted_Issue_Solution.xlsx')

question_metrics_df = evaluate_summaries_at_sentence_level(df, 'Ground_truth_Issue_Labeled', 'Issue_Extracted')
answer_metrics_df = evaluate_summaries_at_sentence_level(df, 'Ground_truth_Solution_Labeled', 'Solution_Extracted')

question_metrics_df.columns = [f'Question_{col}' for col in question_metrics_df.columns]
answer_metrics_df.columns = [f'Answer_{col}' for col in answer_metrics_df.columns]

combined_metrics_df = pd.concat([question_metrics_df, answer_metrics_df], axis=1)

# Compute overall Precision, Recall, F1 scores
mean_question_metrics = question_metrics_df.mean()
mean_answer_metrics = answer_metrics_df.mean()

print("\nMean Precision, Recall, F1 Scores for \033[31mQuestions\033[0m:")
print(mean_question_metrics)

print("\nMean Precision, Recall, F1 Scores for \033[31mAnswers\033[0m:")
print(mean_answer_metrics)



Mean Precision, Recall, F1 Scores for Questions:
Question_precision    0.562250
Question_recall       0.548818
Question_f1           0.553699
dtype: float64

Mean Precision, Recall, F1 Scores for Answers:
Answer_precision    0.540215
Answer_recall       0.516076
Answer_f1           0.525655
dtype: float64
